<a href="https://colab.research.google.com/github/jpmazate90/HorarioGenetico/blob/master/HorarioInteligente_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask-ngrok

In [ ]:
class Docente:

  def __init__(self,carnet,nombre):
    self.carnet = carnet
    self.nombre = nombre

class Curso:
    def __init__(self,codigo,nombre,ciclo,seccion,carrera):
      self.codigo = codigo
      self.nombre = nombre
      self.ciclo = ciclo
      self.seccion = seccion
      self.carrera = carrera

class Salon:
   
    def __init__(self, paramNumeroSalon, paramCapacidad):
      self.numeroSalon = paramNumeroSalon
      self.capacidad = paramCapacidad

class Cromosoma:
  def __init__(self):
    self.lista = []
    self.porcentaje = 100
  

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials

import time

gc = gspread.authorize(GoogleCredentials.get_application_default())

#Captura de datos
docentesWorksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1x8im5OKNCfnfJ3IWCysxqlfX-rpL0fGKLasZV8roaWc/edit#gid=613333572').sheet1
salonesWorksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/19kLciSLwoqPUgMhwnaJkIMVVKaYFBw_exNLBW2-jmI4/edit#gid=1713769866').sheet1
cursosWorksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1PQKKlGwJ3_ai6JITlMjBsOyEOOMzS9ig3ZOfArAPHZI/edit#gid=1737435044').sheet1

# get_all_values gives a list of docentesWorksheet.
docentes = docentesWorksheet.get_all_values()
listaDocentes=[]
del docentes[0]
for row in docentes:
  listaDocentes.append(Docente(row[1],row[2]))

# get_all_values gives a list of docentesWorksheet.
salones = salonesWorksheet.get_all_values()
listaSalones=[]
del salones[0]
for row in salones:
 listaSalones.append(Salon(row[1],row[2]))

# get_all_values gives a list of docentesWorksheet.
cursos = cursosWorksheet.get_all_values()
listaCursos=[]
del cursos[0]
for row in cursos:
  listaCursos.append(Curso(row[1],row[2],row[3],row[4],row[5]))

# Convert to a DataFrame and render.
import pandas as pd
import random
import numpy as np

CANTIDAD_PERIODOS = 6
CANTIDAD_CROMOSOMAS = 20
CANTIDAD_PENALIZACION_R1 = -5
CANTIDAD_PENALIZACION_R2 = -2.5
PORCENTAJE_MINIMO_CROMOSOMA = 50
PORCENTAJE_ACEPTACION_CROMOSOMA = 90
PORCENTAJE_MAXIMO_CROMOSOMA = 100


def crearCromosoma(listadoParaCromosoma):
  cromosoma= Cromosoma()
  for salon in listadoParaCromosoma:
    listado= []
    for periodo in range(CANTIDAD_PERIODOS):  
      listado.append(random.choice(listaCursos))  
    cromosoma.lista.append(listado)
  return cromosoma



def mostrarCromosoma(cromosomaa:Cromosoma):
  contador = 1
  print("Porcentaje de aceptacion del cromosoma: "+str(cromosomaa.porcentaje)+"%")
  for crom in cromosomaa.lista:
    print("-----salon "+str(contador)+"-------")
    for c in crom:
      print(" ###Curso### \n Ciclo: "+c.ciclo  + "\n Codigo: "+c.codigo+"\n Nombre: "+c.nombre+"\n Seccion: "+c.seccion+"\n Carrera: "+c.carrera)
    contador+=1
  
def crearPoblacionInicial():
  arregloCreadoPoblacion = []
  for i in range(CANTIDAD_CROMOSOMAS):
    arregloCreadoPoblacion.append(crearCromosoma(listaSalones))
  return arregloCreadoPoblacion

def mostrarPoblacionInicial(arregloPoblacion):
  contador = 1
  for crom in arregloPoblacion:
    print("*************CROMOSOMA #"+str(contador)+"*************")
    mostrarCromosoma(crom)
    contador +=1

#Funcion para seleccionar cormosomas

def seleccionarCromosomas(cromosomaa:Cromosoma):
  
  porcentajeCr = 0;
  numeroSalon = 1;
  
  for salon in cromosomaa.lista:
    contador = 0
    for c in salon:
      porcentajeCr += compararCromosomaR1(c,numeroSalon,contador,cromosomaa.lista)
      porcentajeCr += compararCromosomaR2(c,contador,salon,numeroSalon)
      contador+=1
    numeroSalon +=1  

  cromosomaa.porcentaje = PORCENTAJE_MAXIMO_CROMOSOMA + porcentajeCr

            

def compararCromosomaR1(cr:Curso,numeroSalonActual,posicionCurso, listadoSalonesCromosoma):
  porcentaje = 0
  numeroSalonLocal = 1
  for salon in listadoSalonesCromosoma:
    if numeroSalonLocal!=numeroSalonActual:
      contador = 0
      for curso in salon:
        if contador == posicionCurso:
          if cr.seccion == curso.seccion and cr.ciclo == curso.ciclo and cr.carrera == curso.carrera:
            print("Error R1: Salon: "+str(numeroSalonActual)+" Curso a comparar: "+cr.nombre,cr.seccion,cr.ciclo, ". Salon:"+ str(numeroSalonLocal)+" Curso con el que se comparo:"+curso.nombre,curso.seccion, curso.ciclo)
            porcentaje+=CANTIDAD_PENALIZACION_R1
        contador+=1
    numeroSalonLocal +=1
  return porcentaje;

def compararCromosomaR2(cr:Curso,posicionCurso, listadoCursos, numeroSalon):
  porcentaje = 0
  contador = 0
  for curso in listadoCursos:
    if contador != posicionCurso:
      if cr.seccion == curso.seccion and cr.ciclo == curso.ciclo and cr.codigo == curso.codigo and cr.carrera == curso.carrera: 
        print("Error R2: Salon: "+str(numeroSalon)+" Curso a comparar: "+cr.nombre,cr.seccion,cr.ciclo, ". Curso con el que se comparo:"+curso.nombre,curso.seccion, curso.ciclo)
        porcentaje+=CANTIDAD_PENALIZACION_R2
    contador+=1
  return porcentaje;


def seleccionarPoblacion(arregloPoblacion):
  contador = 0
  for crom in arregloPoblacion:
    print("*****CROMSOMA "+str(contador)+"****************")
    seleccionarCromosomas(crom)
    contador+=1

# eliminar cromosomas malos
def eliminarCromosomasMalos(porcentajeMinimo, listadoCromosomas):
  retornar = []
  for cromosoma in listadoCromosomas:
    if cromosoma.porcentaje>=porcentajeMinimo:
        retornar.append(cromosoma)
  
  return retornar;


#mutacion
def seleccionarMutacion(arregloMutacion):
  contador = 0
  for crom in arregloMutacion:
    realizarMutacion(crom)
    contador+=1


def realizarMutacion(cromosomaa:Cromosoma):
  numeroSalon = 1;
  for salon in cromosomaa.lista:
    contador = 0
    for c in salon:
      compararMutacionR1(c,numeroSalon,contador,cromosomaa.lista)
      compararMutacionR2(c,contador,salon,numeroSalon)
      contador+=1
    numeroSalon +=1  

def compararMutacionR1(cr:Curso,numeroSalonActual,posicionCurso, listadoSalonesCromosoma):
  numeroSalonLocal = 1
  for salon in listadoSalonesCromosoma:
    if numeroSalonLocal!=numeroSalonActual:
      contador = 0
      for curso in salon:
        if contador == posicionCurso:
          if cr.seccion == curso.seccion and cr.ciclo == curso.ciclo and cr.carrera == curso.carrera:
            curso =  random.choice(listaCursos)
        contador+=1
    numeroSalonLocal +=1

def compararMutacionR2(cr:Curso,posicionCurso, listadoCursos, numeroSalon):
  contador = 0
  for curso in listadoCursos:
    if contador != posicionCurso:
      if cr.seccion == curso.seccion and cr.ciclo == curso.ciclo and cr.codigo == curso.codigo and cr.carrera == curso.carrera:
        curso =  random.choice(listaCursos) 
    contador+=1

def existeUnHorarioAceptable(porcentajeAceptacion, arregloPoblacion):
  for crom in arregloPoblacion:
    if crom.porcentaje >= porcentajeAceptacion:
      return True
  return False

def crossover(listaPoblacionInicial):
  crom = Cromosoma()
  lista = []
  porcentajeHijo = 50
  cont = 0;
  numeroSalones = len(listaSalones)
  mitadNumSalones =  int(numeroSalones/2) - 1
  mitad2NumSalones = mitadNumSalones+1
  if (numeroSalones < 2):
    print("Los salones no son suficientes para realizar el crossover")
  else:
    for c in listaPoblacionInicial:
      if (cont < len(listaPoblacionInicial)-1):
        siguiente = cont+1
        c2 = listaPoblacionInicial[siguiente]
        hijo1 = Cromosoma()
        salonesHijo1 = []
        for i in range(0,mitadNumSalones+1):
          salonesHijo1.append(c.lista[i])
        for j in range(mitad2NumSalones,numeroSalones):
          salonesHijo1.append(c2.lista[j])
        hijo1.lista = salonesHijo1
        hijo1.porcentaje = porcentajeHijo
        lista.append(hijo1)

        hijo2 = Cromosoma()
        salonesHijo2 = []
        for i in range(mitad2NumSalones,numeroSalones):
          salonesHijo2.append(c.lista[i])
        for j in range(0,mitadNumSalones+1):
          salonesHijo2.append(c2.lista[j])
        hijo2.lista = salonesHijo2
        hijo2.porcentaje = porcentajeHijo
        lista.append(hijo2)
    for i in lista:
      listaPoblacionInicial.append(i)


#main
poblacionInicial= crearPoblacionInicial()
seleccionarPoblacion(poblacionInicial)
poblacionInicial = eliminarCromosomasMalos(PORCENTAJE_MINIMO_CROMOSOMA ,poblacionInicial)

existeHorarioAceptado = existeUnHorarioAceptable(PORCENTAJE_ACEPTACION_CROMOSOMA,poblacionInicial)
existeHorarioAceptado = False
contador = 1
subirPorcentaje = PORCENTAJE_MINIMO_CROMOSOMA
while (not existeHorarioAceptado and  len(poblacionInicial)>0):
  print(" ########## Se van a realizar procesos de mejora ------------------")
  time.sleep(2)
  crossover(poblacionInicial)
  print(" ---------------------------------------- ALTO!! ------------------------------ ")
  time.sleep(2)
  seleccionarPoblacion(poblacionInicial)
  existeHorarioAceptado = existeUnHorarioAceptable(PORCENTAJE_ACEPTACION_CROMOSOMA,poblacionInicial)

  #subirPorcentaje +=2
  #poblacionInicial = eliminarCromosomasMalos(subirPorcentaje ,poblacionInicial)
  if (not existeHorarioAceptado):
    seleccionarMutacion(poblacionInicial)
    seleccionarPoblacion(poblacionInicial)
    existeHorarioAceptado = existeUnHorarioAceptable(PORCENTAJE_ACEPTACION_CROMOSOMA,poblacionInicial)
  
  subirPorcentaje +=2
  poblacionInicial = eliminarCromosomasMalos(subirPorcentaje ,poblacionInicial)
  
  print ("          &&&&&&&&& REALIZANDO ITERACION #",contador,"&&&&&&&&&&&&&&&&&&&&&&&&")
  contador +=1
  if (contador>10):
    existeHorarioAceptado = True


poblacionInicial = eliminarCromosomasMalos(PORCENTAJE_ACEPTACION_CROMOSOMA ,poblacionInicial)
#seleccionarPoblacion(poblacionInicial)


*****CROMSOMA 0****************
Error R1: Salon: 1 Curso a comparar: ORIENTACION Y LIDERAZGO B 1 . Salon:2 Curso con el que se comparo:QUIMICA 1 B 1
Error R1: Salon: 1 Curso a comparar: MATEMATICA INTERMEDIA 1 B 3 . Salon:2 Curso con el que se comparo:FISICA 1 B 3
Error R1: Salon: 1 Curso a comparar: LEGISLACION A 5 . Salon:5 Curso con el que se comparo:CONTABILIDAD 1 A 5
Error R1: Salon: 2 Curso a comparar: QUIMICA 1 B 1 . Salon:1 Curso con el que se comparo:ORIENTACION Y LIDERAZGO B 1
Error R1: Salon: 2 Curso a comparar: FISICA 1 B 3 . Salon:1 Curso con el que se comparo:MATEMATICA INTERMEDIA 1 B 3
Error R1: Salon: 2 Curso a comparar: MATEMATICA BASICA 1 C 1 . Salon:3 Curso con el que se comparo:ORIENTACION Y LIDERAZGO C 1
Error R1: Salon: 3 Curso a comparar: ORIENTACION Y LIDERAZGO C 1 . Salon:2 Curso con el que se comparo:MATEMATICA BASICA 1 C 1
Error R2: Salon: 5 Curso a comparar: DEPORTES 1 B 1 . Curso con el que se comparo:DEPORTES 1 B 1
Error R1: Salon: 5 Curso a comparar: CONT

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import jsonify
import json

app = Flask(__name__)
run_with_ngrok(app)

#@app.route('/')
#def helloWorld():
#  return 'Hello,World'

def crearJsonListCurso(listadoDeCursos):
  cursos ={}
  cursos['curso']=[]
  for curso in listadoDeCursos:
      curso ={
          'codigo':curso.codigo,
          'nombre':curso.nombre,
          'ciclo':curso.ciclo,
          'seccion':curso.seccion,
          'carrera':curso.carrera
      }
      cursos['curso'].append(curso)
  return cursos

def crearJsonList(listadoDeSalones):
  salones = {}
  salones['salon'] =[]
#  salones['numeroDeSalon']=[]
  numeroDeSalon=1;
  for salon in listadoDeSalones:
#    salones['numeroDeSalon'].append(numeroDeSalon)
    salones['salon'].append({
        'numeroDeSalon':numeroDeSalon,
        'cursos':crearJsonListCurso(salon)        
    })
    numeroDeSalon+=1
  return salones


@app.route('/')
def getPoblacion():
  cromosomas = {}
  #cromosomas['porcentaje']=[]
  #cromosomas['salones']=[]
  cromosomas['cromosoma'] = []
  for crom in poblacionInicial:

    #Ya funciono 
    #cromosomas['porcentaje'].append({'porcentaje':crom.porcentaje})
    #cromosomas['salones'].append(crearJsonList(crom.lista))
    cromosomas['cromosoma'].append({
          'salones' : crearJsonList(crom.lista),
          'porcentaje' : crom.porcentaje
       })

  return cromosomas


if __name__ == '__main__':
  app.run()  


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3a890e0f6e60.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [05/May/2021 10:01:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 10:01:34] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [05/May/2021 10:01:34] "GET / HTTP/1.1" 200 -
